## 是否可以求导 requires_grad=True
## x是直接创建的，所以它没有grad_fn，称为叶子节点

In [1]:
import torch


x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


## 运算名称 grad_fn=AddBackward0

In [3]:

y = x + 2
print(y)
print(y.grad_fn)

print(x.is_leaf, y.is_leaf)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
True False


In [4]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


In [5]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)


False
True


## out = z.mean()
## out关于x的梯度 d(out)/dx
## 此为标量的梯度

In [6]:
out.backward()
print(x.grad)


tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


## grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零。

In [7]:
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)


tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


## 向量梯度是个雅可比矩阵
## torch.autograd这个包就是用来计算一些雅克比矩阵的乘积的
## 不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量。所以必要时要把张量通过将所有张量的元素加权求和的方式转换为标量
## y.backward(w)的含义是：先计算l = torch.sum(y * w)，则l是个标量，然后求l对自变量x的导数

In [17]:
x = torch.tensor([1.0,2.0,3.0,4.0],requires_grad=True)
y = 2 * x
z = y.view(2,2)
print(z)

v = torch.tensor([[1.0,0.1],[0.01,0.001]],dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)
tensor([2.0000, 0.2000, 0.0200, 0.0020])


## 由于y2的定义是被torch.no_grad():包裹的，所以与y2有关的梯度是不会回传的，只有与y1有关的梯度才会回传，即x^2对x的梯度。

In [ ]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

y3.backward()
print(x.grad)

## 想要修改tensor的数值，但是又不希望被autograd记录（即不会影响反向传播），那么我么可以对tensor.data进行操作

In [10]:
x = torch.ones(1, requires_grad=True)
print(x.data)
print(x.data.requires_grad)

y = 2 * x
x.data *= 100

y.backward()
print(x)
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
